## VAR Automation

#### Description:
###### This file contains the code for VAR automation, along with csv's for intermediate results such as portfolio holdings, price changes for BOTH AAVE AND COMP

###### Author: Nishchal Gaba
###### Created on: January 13, 2022

#### AAVE

In [49]:
import os
import json
import boto3
import requests
import numpy as np
import pandas as pd
from io import StringIO
from datetime import timedelta
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


import matplotlib.pyplot as plt

In [50]:
# Coingecko data fetcher
def getCoinGeckoHistoricalData(_token):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _token(str): Token Symbol (actual token will be fetched from token_names dictionary)
            
        Returns::
            _resDf(pandas dataframe): Returns the dates (yyyy-mm-dd) and prices (USD) in a dataframe
    '''
    try:
        # Making the coingecko historical API request
        coinGeckoTokenCall = token_names[_token]
        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coinGeckoTokenCall)+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        
        # Converting the result to JSON
        resJSON = request.json()
        
        # Creating empty dataframe
        resDf = pd.DataFrame(resJSON['prices'], columns = ["timestamp", "price"])
        
        # We divide by 1000 to convert the default milliseconds from CoinGecko to seconds
        # Get the human readable date from timestamp
        resDf['timestamp'] = resDf['timestamp'].apply(lambda x: str(datetime.fromtimestamp(x/1000).date()))
        
        # Return the resultant dataframe
        return resDf
        
        
    except Exception as e:
        print("Exception occured while fetching Coingecko Data")
        return str(e)

In [51]:
# AAVE V2 data for a particular date
def aaveV2HistoricalData(_date):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _date(str): date for which data is needed in (%m-%d-%Y) format
            
        Returns::
            _resDf(pandas dataframe): Token, CurrentPrice, availableLiquidity, decimals,
                                        totalLiquidity, utilizationRate, Assets
    '''
    
    try:
        # Starting Index
        ind = 0
        
        # Calling the Aave V2
        aaveV2res = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
        resDf = pd.DataFrame(columns=['Token', 'CurrentPrice', 'availableLiquidity','decimals', 'totalLiquidity','utilizationRate','Assets'])
        
        # Iterating through tokens in AaveV2 API
        for token in aaveV2res:
            try:
                resDf.loc[ind] = [token['symbol']] + [float(token['referenceItem']['priceInUsd'])] + \
                                [float(token['availableLiquidity'])] + [float(token['decimals'])]+ \
                                [float(token['totalLiquidity'])]+ [float(token['referenceItem']['utilizationRate'])] + \
                                [float(token['totalDebt'])]

                ind += 1

            except Exception as e:
                print("**", token, e) #str(asset), str(price_mean), str(price_std), str(cap_mean),str(cap_std), str(vol_mean) , str(vol_std))  
    #             errors.append(token)
                # transactionResponse["error"] = True
                print("Exception occured while fetching Aave V2 Token wise Data")
                return str(e)


        # totalLiquidity == Market
        resDf['totalLiquidity']       = resDf['totalLiquidity'] * resDf['CurrentPrice']
        resDf['Assets']               = resDf['Assets'] * resDf['CurrentPrice']
        resDf['Position']             = resDf['Assets'] - resDf['totalLiquidity']
        resDf['Position($,Billion)']  = resDf['Position'] / pow(10, 9)


        # Return the resultant dataframe
        return resDf
        
    except Exception as e:
        print("Exception occured while fetching Aave V2 API Data")
        return str(e)

In [52]:
# Coingecko Mapping for each token name to fetch historical prices
names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        "REN"   : "ren",
        "RENFIL": "renfil",
        "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }
# Funtion to give historical price changes for each token mentioned in Names Dictionary
def change():
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        P    = []
        if request.status_code != 200:
            data = {}
            data['prices'] = WBTC[ -LOOK_BACK: ]
            for day in data['prices']:
                P.append(day)
        else:   
            data = request.json()
            tt   = []
            

            if len(data['prices']) < LOOK_BACK:
                req = LOOK_BACK - len(data['prices'])
                addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
                data['prices'] = addition + data['prices']
                track[asset] = req
            else:
                data['prices'] = data['prices'][-LOOK_BACK:]

            for day in data['prices']:
                tt.append(day[0]/1000)
                P.append(day[1])
                
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    
    print("PRICES FETECHED")
    
    unknown_assets = ['PAX']
    for asset in list(names.keys()):#[:LIMIT]:
        try:
            change = []
            diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
            
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(10)'] = change

            diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
            change = []
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(1)'] = change  
            
        except Exception as e:
            unknown_assets.append(asset)
            print("Error ", asset, e)
    
    for asset in unknown_assets:
        df[asset+'_%change(1)']  = df['WBTC_%change(1)'] 
        df[asset+'_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(1)'] = df['WBTC_%change(1)'] 
        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df[:-10]
    return df
# change_df = change()

In [53]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [54]:
# query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

In [55]:
# V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)

In [56]:
# Required Date
# By default set to current date
# requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate = '12-16-2021' #"01-10-2022" #mm-dd-yyyy

In [57]:
def VARDate(_date):
    '''
        Gets the VAR results, AaveV2 for a particular position and price change dataframe
        
        Args::
            _date(str): Date in "%m-%d-%Y" format
        
        Returns::
            aaveV2Res(pandas dataframe): AaveV2 Historical Data for the given date
            df (pandas dataframe): Price change dataframe
    '''
    # Aave V2 Data for particular date
    aaveV2Res = aaveV2HistoricalData(_date)
    
    # Price Change Dataframe
    df = change()
    
    # Flipping the date format
    reqDate = datetime.strptime(_date, '%m-%d-%Y').strftime('%Y-%m-%d')
    
    # Converting str to datetime
    df['date']= pd.to_datetime(df['date'])
    # Remove dates above the required date
    mask = (df['date'] <= reqDate)
    df = df.loc[mask]
    # Converting the date column back to str
    df['date']=df['date'].astype(str)
    
    REQUIRED_1 = []
    REQUIRED_10 = []
    historicalVAR = pd.DataFrame(columns=['date', 'VAR_10', 'VAR_1'])
    hist_ind = 0
    
    cols     = aaveV2Res['Token']
    FINAL_DF = aaveV2Res[['Position($,Billion)']].T
    FINAL_DF.columns = cols
    # Sliding Window Var Calculator
    VAR      = []
    VAR_DATE = []
    errors   = []
    for ind, ROW in enumerate(FINAL_DF.iterrows()):
        REQUIRED_10 = []
        for required_row in df[ind:ind+365].iterrows():
            SUM_10  = []
            for key in ROW[1].keys():
                if isinstance(ROW[1][key], list):
                    ROW[1][key] = 0
                SUM_10.append((float(ROW[1][key]) * required_row[1][key+'_%change(10)'])/100)
            REQUIRED_10.append(sum(SUM_10))
        VAR.append(sorted(REQUIRED_10, key=float)[3])
        VAR_DATE.append(df[ind:ind+365].at[REQUIRED_10.index(sorted(REQUIRED_10, key=float)[3]) + ind, 'date'])
    
    testdf = pd.DataFrame(columns=['SYMBOL', 'totalLiquidity', 'totalDebt'])
    ind = 0
    
    AAVE_API_RESULT  = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
    
    for i in AAVE_API_RESULT:
        testdf.loc[ind] = [i['symbol']] + [float(i['totalLiquidity']) * float(i['referenceItem']['priceInUsd']) ]\
                    + [float(i['totalDebt']) * float(i['referenceItem']['priceInUsd'] )] #+ [i['referenceItem']['priceInUsd'] * ]
        ind += 1

    TOTAL_LIABILITIES    = aaveV2Res['totalLiquidity'].sum()
    TOTAL_ASSETS         = aaveV2Res['Assets'].sum()
    VAR_10_DAY_99_P      = VAR
    VAR_DATE_10_DAY_99_P = VAR_DATE

#     RELATIVE_VAR_ASSETS = str(VAR * pow(10,9) / TOTAL_ASSETS)
#     RELATIVE_VAR_LIABILITIES = str(VAR * pow(10,9) / TOTAL_LIABILITIES)
    #TOTAL_ASSETS
    #TOTAL_LIABILITIES
    return aaveV2Res, df, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

In [58]:
# aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)
# Date in %m-%d-%Y format
aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'


/Users/rahulkanojia/miniconda3/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
VAR_10_DAY_99_P

[-4.450595218045071]

In [60]:
VAR_DATE_10_DAY_99_P

['2021-02-08']

In [61]:
VAR_DATE_10_DAY_99_P

['2021-02-08']

In [62]:
# Token change in price for AAVE VAR Date
idx = priceChangeDf.index[priceChangeDf['date'] == VAR_DATE_10_DAY_99_P[0]].tolist()
priceChangeVARDATE = priceChangeDf.loc[idx]
# _index = priceChangeDf.index[mask]
# priceChangeDf[_index]

In [63]:
priceChangeVARDATE

,timestamp,DAI_price,DAI_%change(10),GUSD_price,GUSD_%change(10),SUSD_price,SUSD_%change(10),TUSD_price,TUSD_%change(10),USDC_price,...,MANA_%change(1),RENFIL_%change(1),ZRX_%change(1),PAX_%change(1),PAX_%change(10),SUSD_%change(1),ENJ_%change(1),REN_%change(1),SNX_%change(1),date
344,1.612742e+09,1.000595,0.413431,0.988156,-2.40554,"[1612742400000, 38884.62473910085]",17.70038,0.996031,0.304678,0.995688,...,3.502594,1.889281,-4.630101,-1.274438,17.70038,-1.274438,-1.274438,-1.274438,-1.274438,2021-02-08


In [64]:
# Contribution to VAR
# -1 * (TOKEN_PRICE * TOKEN_POSITION * TOKEN%CHANGE)

In [65]:
# PLATFORM, TOKEN, POSITION_ON_INPUT_DATE, POSITION_in_B_ON_INPUT_DATE, TOKEN_%_CHANGE_ON_VAR_DATE, CONTRIBTION TO VAR

In [66]:
column_names = ["PLATFORM", "TOKEN", "POSITION_ON_INPUT_DATE", "POSITION_ON_INPUT_DATE_BILLIONS",
               "TOKEN_%_CHANGE_ON_VAR_DATE", "CONTRIBUTION_TO_VAR"]
resultDf = pd.DataFrame(columns = column_names)

In [67]:
# Itearing through the position in Aave
# for token in aaveV2Res["Token"]:
for idx, row in aaveV2Res.iterrows():
    try:
        temp = []
        
        temp.append("AAVE")
        temp.append(str(aaveV2Res["Token"][idx]))
        temp.append(aaveV2Res["Position"][idx])
        temp.append(aaveV2Res["Position($,Billion)"][idx])
        temp.append(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])
        temp.append(float(-1 * float(aaveV2Res["Position($,Billion)"][idx]) * float(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])))
        
        
        
        # Add the values to the latest row
        df_length = len(resultDf)
        resultDf.loc[df_length] = temp
        
    except:
        print("Exception occured for the token: "+str(token))

# print(temp)

In [68]:
# Sample Final Res
resultDf

,PLATFORM,TOKEN,POSITION_ON_INPUT_DATE,POSITION_ON_INPUT_DATE_BILLIONS,TOKEN_%_CHANGE_ON_VAR_DATE,CONTRIBUTION_TO_VAR
0,AAVE,TUSD,-4.879721e+07,-0.048797,0.304678,0.014867
1,AAVE,RAI,-6.177154e+06,-0.006177,17.700380,0.109338
2,AAVE,GUSD,-2.911591e+06,-0.002912,-2.405540,-0.007004
3,AAVE,YFI,-3.678891e+07,-0.036789,3.559561,0.130952
4,AAVE,BAT,-4.812508e+06,-0.004813,32.820921,0.157951
5,AAVE,MANA,-4.231312e+07,-0.042313,48.881258,2.068319
6,AAVE,DPI,-4.451598e+07,-0.044516,37.968346,1.690198
7,AAVE,UNI,-3.914723e+07,-0.039147,31.217438,1.222076
8,AAVE,WBTC,-1.257739e+09,-1.257739,17.700380,22.262463
9,AAVE,REN,-2.420904e+07,-0.024209,17.700380,0.428509


### Compound Code

In [1]:
# Imports
import os
import json
import boto3
import requests
import numpy as np
import pandas as pd
from io import StringIO
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


import matplotlib.pyplot as plt

In [ ]:
for ctoken in res.json()['cToken']:
    break

In [2]:
requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate = '12-16-2021' #"01-10-2022" #mm-dd-yyyy

In [40]:
class Compound_date_VAR():
    def __init__(self, REQUIRED_DATE):
        self.names = {"DAI"  : 'dai',
            "TUSD"  : 'true-usd',
            "USDC"  : 'usd-coin',
            "USDT"  : "tether",
            "ETH"   : "ethereum",
            "LINK"  : "chainlink",
            "MKR"   : "maker",
            "UNI"   : "uniswap",
            "WBTC"  : "wrapped-bitcoin",
            "WBTC2"  : "wrapped-bitcoin",
            "SUSHI" :"sushi",  
            "YFI"   : "yearn-finance",
            "AAVE"  : "aave",
            "BAT"   : "basic-attention-token",
            "ZRX"   : "0x",
            "REP"   : "augur",
            "SAI"   : "sai",
            }
        self.priceChangeDf = None
        self.FINAL_DF = None
        self.VAR = None
        self.VAR_DATE =None
        self.REQUIRED_DATE = REQUIRED_DATE
        
    def calculatePriceChangeDf(self):   
        columns = []
        columns.append('timestamp')
        for key in list(self.names.keys()):
            columns.append(key+'_price')
            columns.append(key+'_%change(10)')
        df        = pd.DataFrame(columns = columns)
        LOOK_BACK = 375 * 2
        #LIMIT     = 5
        WBTC      = []
        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(self.names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data      = request.json()
        WBTC      = data['prices']
        tt        = []
        P         = []
        for day in data['prices'][-LOOK_BACK:]:
            tt.append(day[0]/1000)
            P.append(day[1])
        df['WBTC_price'] = P
        df['timestamp']  = tt
        track = {}
        for asset in list(self.names.keys()):#[:LIMIT]:
            if asset == 'WBTC':
                continue
            request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(self.names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
            data = request.json()
            tt = []
            P = []
            # print("DEBUG 1 ", data)
            if len(data['prices']) < LOOK_BACK:
                req = LOOK_BACK - len(data['prices'])
                addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
                data['prices'] = addition + data['prices']
                track[asset] = req
            else:
                data['prices'] = data['prices'][-LOOK_BACK:]

            for day in data['prices']:
                tt.append(day[0]/1000)
                P.append(day[1])
            df[asset+'_price'] = P
        for asset in list(self.names.keys()):#[:LIMIT]:

            diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
            change = []
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(10)'] = change

            diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
            change = []
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(1)'] = change        
        for key in track.keys():
            df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

        df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
        df  = df.iloc[::-1]
        df  = df.reset_index()
        df = df.drop(['index'], axis=1)
        df = df[1:-10]
        
        reqDate = datetime.strptime(self.REQUIRED_DATE, '%m-%d-%Y').strftime('%Y-%m-%d')
        df['date']= pd.to_datetime(df['date'])
        mask = (df['date'] <= reqDate)
        df = df.loc[mask]
        df['date']=df['date'].astype(str)
        self.priceChangeDf  = df      
    def getPositionData(self):
        ind  = 0 
        res  = requests.get("https://api.compound.finance/api/v2/ctoken")
        df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
        for ctoken in tqdm(res.json()['cToken']):
            try:
                ind += 1
                for key in ctoken.keys():
                    try:
                        df.at[ind,key ] = ctoken[key]['value']
                    except:
                        df.at[ind,key ] = ctoken[key]

                address   = ctoken['token_address']
                request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(self.names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
                prices_df = pd.DataFrame(columns=['timestamp', 'price'])
                timestamp = []
                prices    = []

                for req in request.json()['prices']:
                    timestamp.append(int(float(req[0])/1000))
                    prices.append(req[1])

                prices_df['timestamp'] = timestamp
                prices_df['price']     = prices

                date= []
                for time in timestamp:
                    date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
                prices_df['date']  = date

                datestamp = self.REQUIRED_DATE #mm-dd-yyyy
                max_timstamp = int(datetime.strptime(datestamp, '%m-%d-%Y').timestamp()) + 86400
                SUPPLY_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
                BORROW_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
                DF = pd.DataFrame()
                BIND = 0
                SIND = 0
                for _ in range(1):
                    min_timestamp = max_timstamp  -  3 * 86400 
                    res = requests.get("https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=50" % ( address,min_timestamp, max_timstamp));    
                    max_timstamp = max_timstamp   - 3 * 86400 
                    temp = res.json()


                    for tem in temp['total_borrows_history']:
                        BORROW_DF.loc[BIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]                     
                        BIND += 1


                    for tem in temp['total_supply_history']:
                        SUPPLY_DF.loc[SIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]
                        SIND += 1                      

                BORROW_DF         = BORROW_DF.sort_values(['block_timestamp'])
                BORROW_DF['DATE'] = BORROW_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
                BORROW_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)

                SUPPLY_DF         = SUPPLY_DF.sort_values(['block_timestamp'])
                SUPPLY_DF['DATE'] = SUPPLY_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
                SUPPLY_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)

                SUPPLY_DF['supply_tokens'] = pd.to_numeric(SUPPLY_DF['value']) *  float(temp['exchange_rates'][-1]['rate'])


                BORROW_DF          = BORROW_DF[::-1]
                SUPPLY_DF          = SUPPLY_DF[::-1]
                DF['timestamp']    = BORROW_DF['block_timestamp']
                DF['BORROW']       = BORROW_DF['value']
                DF['SUPPLY']       = SUPPLY_DF['supply_tokens']
                DF['DATE']         = BORROW_DF['DATE']
                DF['SUPPLY_VALUE'] = SUPPLY_DF['value']
                
                assigned_price = []
                for indW in DF['DATE'] :
                    a = prices_df[prices_df['date']==str(indW)]
                    assigned_price.append(a['price'].values[0])

                DF['prices'] = assigned_price
                DF['total_supply'] =  DF['SUPPLY'] * DF['prices']
                DF['total_borrow'] =  pd.to_numeric(DF['BORROW']) * pd.to_numeric(DF['prices'])
                DF['POSITION']     =  DF['total_borrow'] - DF['total_supply']
                DF['POSITION($)']  =  DF['POSITION'] / pow(10,9)
                DF.to_csv('COMPOUND_DATA/DF_'+ctoken['symbol'][1:] + '.csv')
                #print(ctoken['symbol'], DF.shape)
            except Exception as e:
                #print("ERROR ", e)
                pass                              
    def computePositionData(self):
        assert os.path.exists('COMPOUND_DATA/')
        fils = os.listdir('COMPOUND_DATA/')
        cols = []
        for fil in fils:
            try:
                aa = fil.split('.')[0].split('_')
                cols.append(aa[1])
            except:
                pass
        FINAL ={}
        for key in cols:
            try:
                df = pd.read_csv('COMPOUND_DATA/'+'DF_'+key+'.csv')
                df = df.sort_values(['timestamp'])
                df = df[::-1]
                FINAL[key] = {'Assets': df['POSITION($)'],
                              'date':df['DATE'],
                              'MARKET':df['total_supply'],
                              'BORROW':df['total_borrow']
                             }

            except Exception as e:
                print(e, key)

        ff = []
        for f in FINAL['ETH']['date']:
            ff.append(f)

        FINAL_DF = pd.DataFrame(columns= cols)
        ind = 0
        for i in range(len(ff)):
            assets = []
            for col in cols:
                try:
                    assets.append(FINAL[col]['Assets'][i])
                except:
                    assets.append([])
            FINAL_DF.loc[ind] = assets
            ind += 1

        FINAL_DF['DATE']      = ff[:100]
        FINAL_DF['timestamp'] = FINAL_DF['DATE'].apply(lambda x: pd.Timestamp(x).timestamp())
        FINAL_DF              = FINAL_DF.sort_values('timestamp')
        FINAL_DF              = FINAL_DF[::-1]
        
        reqDate = datetime.strptime(self.REQUIRED_DATE, '%m-%d-%Y').strftime('%Y-%m-%d')
        FINAL_DF['DATE']= pd.to_datetime(FINAL_DF['DATE'])
        mask = (FINAL_DF['DATE'] == reqDate)
        FINAL_DF = FINAL_DF.loc[mask]
        FINAL_DF['DATE']=FINAL_DF['DATE'].astype(str)
        
        self.FINAL_DF = FINAL_DF
        
    def getVAR(self):
        if self.FINAL_DF is None:
            self.getPositionData()
            self.computePositionData()
        if self.priceChangeDf is None:
            self.calculatePriceChangeDf()
            
        VAR = []
        VAR_DATE = []
        for ind, ROW in enumerate(self.FINAL_DF.iterrows()):
            REQUIRED_10 = []
            for required_row in self.priceChangeDf[ind:ind+365].iterrows():
                SUM_10 = []
                for key in ROW[1].keys()[:-3]:
                    if key == 'WBTC2':
                        key = 'WBTC'
                    if isinstance(ROW[1][key], list):
                        ROW[1][key] = 0
                    SUM_10.append((float(ROW[1][key]) * required_row[1][key+'_%change(10)'])/100)
                REQUIRED_10.append(sum(SUM_10))
            VAR.append(sorted(REQUIRED_10, key=float)[3])
            VAR_DATE.append(self.priceChangeDf[ind:ind+365].at[REQUIRED_10.index(sorted(REQUIRED_10, key=float)[3]) + ind, 'date'])
        self.VAR = VAR
        self.VAR_DATE = VAR_DATE
        
    def getResult(self):
        idx = self.priceChangeDf.index[self.priceChangeDf['date'] == self.VAR_DATE[0]].tolist()
        self.priceChangeVARDATE = self.priceChangeDf.loc[idx]
        column_names = ["PLATFORM", "TOKEN","POSITION_ON_INPUT_DATE", "POSITION_ON_INPUT_DATE_BILLIONS",
               "TOKEN_%_CHANGE_ON_VAR_DATE", "CONTRIBUTION_TO_VAR"]
        resultDf = pd.DataFrame(columns = column_names)
        FINAL_DF_T = self.FINAL_DF.T
        FINAL_DF_T = FINAL_DF_T.reset_index()
        FINAL_DF_T.columns = ['Token',"Position($,Billion)"]#, 'Position' ]
        FINAL_DF_T['Position'] = FINAL_DF_T['Position($,Billion)'] * pow(10,9)
        
        # Itearing through the position in Compound
        for idx, row in FINAL_DF_T[:-2].iterrows():
            try:
                temp = []
                temp.append("COMPOUND")
                temp.append(str(FINAL_DF_T["Token"][idx]))
                temp.append(FINAL_DF_T["Position"][idx])
                temp.append(FINAL_DF_T["Position($,Billion)"][idx])
                temp.append(self.priceChangeVARDATE[str(FINAL_DF_T["Token"][idx])+"_%change(10)"].iloc[0])
                temp.append(float(-1 * float(FINAL_DF_T["Position($,Billion)"][idx]) * float(self.priceChangeVARDATE[str(FINAL_DF_T["Token"][idx])+"_%change(10)"].iloc[0])))
                df_length = len(resultDf)
                resultDf.loc[df_length] = temp

            except:
                print("Exception occured for the token: "+str(token))

        return resultDf
        

In [41]:
aa = Compound_date_VAR(requiredDate)

In [42]:
aa.calculatePriceChangeDf()

/Users/rahulkanojia/miniconda3/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
aa.getPositionData()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:39<00:00,  2.07s/it]


In [44]:
aa.computePositionData()

/Users/rahulkanojia/miniconda3/envs/env/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


In [45]:
aa.getVAR()

In [46]:
resultDF_Compound = aa.getResult()

MKR
UNI
SUSHI
DAI
USDC
USDT
ZRX
YFI
TUSD
LINK
ETH
WBTC
WBTC2
BAT
AAVE
REP


,PLATFORM,TOKEN,POSITION_ON_INPUT_DATE,POSITION_ON_INPUT_DATE_BILLIONS,TOKEN_%_CHANGE_ON_VAR_DATE,CONTRIBUTION_TO_VAR
0,COMPOUND,MKR,-1.701015e+06,-0.001701,61.525970,0.104657
1,COMPOUND,UNI,-1.228164e+08,-0.122816,19.032892,2.337551
2,COMPOUND,SUSHI,-5.034489e+05,-0.000503,30.821792,0.015517
3,COMPOUND,DAI,-8.702621e+08,-0.870262,-0.252673,-0.219892
4,COMPOUND,USDC,-1.115693e+09,-1.115693,0.012229,0.013644
5,COMPOUND,USDT,-2.885674e+08,-0.288567,-0.375969,-0.108492
6,COMPOUND,ZRX,-9.443503e+07,-0.094435,164.814259,15.564239
7,COMPOUND,YFI,-7.554223e+05,-0.000755,48.442986,0.036595
8,COMPOUND,TUSD,-3.991963e+07,-0.039920,-0.007647,-0.000305
9,COMPOUND,LINK,-3.941646e+07,-0.039416,29.048404,1.144985


In [48]:
print("VAR ", aa.VAR[0])

VAR  -4.348821404852034
